In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import tensorflow as tf
import keras
from keras.layers import Dense, Input
from keras import Sequential
from keras.losses import MeanSquaredError, BinaryCrossentropy
from keras.activations import sigmoid
import logging

logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [2]:
X_train = np.array([[1.0], [2.0]], dtype=np.float32)  # (size in 1000 square feet)
Y_train = np.array([[300.0], [500.0]], dtype=np.float32)  # (price in 1000s of dollars)

In [ ]:
fix, ax = plt.subplots(1, 1)
ax.scatter(X_train, Y_train, marker="x", c="r", label="Data Points")
ax.set_ylabel("Price (in 1000s of dollars)")
ax.set_xlabel("Size (1000 sqft)")
ax.legend()
plt.show()

In [ ]:
linear_layer = Dense(units=1, activation="linear")

In [ ]:
linear_layer.get_weights()

In [ ]:
a1 = linear_layer(X_train[0].reshape(-1, 1))
print(a1)

In [ ]:
w, b = linear_layer.get_weights()
print(f"w = {w}, b = {b}")

In [ ]:
set_w = np.array([[200]])
set_b = np.array([100])
linear_layer.set_weights([set_w, set_b])
print(linear_layer.get_weights())

In [ ]:
a1 = linear_layer(X_train[0].reshape(-1, 1))
print(a1)
alin = X_train[0].reshape(-1, 1) @ set_w + set_b
print(alin)

In [ ]:
prediction_tf = linear_layer(X_train)
prediction_np = X_train @ set_w + set_b
print(prediction_tf)
print(prediction_np)

In [ ]:
def plt_linear(X_train, Y_train, prediction_tf, prediction_np):
    fig, ax = plt.subplots(1, 2, figsize=(16, 4))
    ax[0].scatter(X_train, Y_train, marker="x", c="r", label="Data Points")
    ax[0].plot(X_train, prediction_tf, c="b", label="model output")
    ax[0].text(1.6, 350, r"y=$200 x + 100$", c="magenta")
    ax[0].set_ylabel("Price (in 1000s of dollars)")
    ax[0].set_xlabel("Size (1000 sqft)")
    ax[0].legend()
    ax[0].set_title("Tensorflow prediction")

    ax[1].scatter(X_train, Y_train, marker="x", c="r", label="Data Points")
    ax[1].plot(X_train, prediction_np, c="b", label="model output")
    ax[1].text(1.6, 350, r"y=$200 x + 100$", c="magenta")
    ax[1].set_ylabel("Price (in 1000s of dollars)")
    ax[1].set_xlabel("Size (1000 sqft)")
    ax[1].legend()
    ax[1].set_title("Numpy prediction")

    plt.show()

In [ ]:
plt_linear(X_train, Y_train, prediction_tf, prediction_np)

In [ ]:
X_train = np.array([0.0, 1, 2, 3, 4, 5], dtype=np.float32).reshape(-1, 1)  # 2-D Matrix
Y_train = np.array([0, 0, 0, 1, 1, 1], dtype=np.float32).reshape(-1, 1)  # 2-D Matrix

In [ ]:
pos = Y_train == 1
neg = ~pos

fig, ax = plt.subplots(1, 1, figsize=(4, 3))
ax.scatter(X_train[pos], Y_train[pos], marker="x", s=80, c="r", label="y=1")
ax.scatter(
    X_train[neg],
    Y_train[neg],
    marker="o",
    s=90,
    label="y=0",
    facecolors="none",
    edgecolors="b",
    lw=3,
)
ax.set_ylim(-0.08, 1.1)
ax.set_ylabel("y")
ax.set_xlabel("x")
ax.set_title("one variable plot")
ax.legend()
plt.show()

In [ ]:
# model = Sequential(
#     [tf.keras.layers.Dense(1, input_dim=1, activation="sigmoid", name="L1")]
# )
model = Sequential([Input(shape=(1,)), Dense(1, activation="sigmoid", name="L1")])

In [ ]:
model.summary()

In [ ]:
logistic_layer = model.get_layer("L1")
w, b = logistic_layer.get_weights()
print(w, b)
print(w.shape, b.shape)

In [ ]:
set_w = np.array([[2]])
set_b = np.array([-4.5])
logistic_layer.set_weights([set_w, set_b])
print(logistic_layer.get_weights())

In [ ]:
a1 = model.predict(X_train[0].reshape(-1, 1))
print(a1)

In [ ]:
def sigmoidnp(z):
    z = np.asarray(z, dtype=np.float32)
    return np.where(z >= 0, 1 / (1 + np.exp(-z)), np.exp(z) / (1 + np.exp(z)))

In [ ]:
alog = sigmoidnp(set_w @ X_train[0].reshape(-1, 1) + set_b)
print(alog)

In [ ]:
def plt_prob_1d(ax, fwb):
    """plots a decision boundary but include shading to indicate the probability"""
    # setup useful ranges and common linspaces
    x_space = np.linspace(0, 5, 50)
    y_space = np.linspace(0, 1, 50)
    z = np.zeros((len(y_space), len(x_space)))
    for i in range(x_space.shape[0]):
        x = np.array([[x_space[i]]])
        z[:, i] = fwb(x).item()
    cmap = plt.get_cmap("Blues")
    pcm = ax.pcolormesh(
        x_space,
        y_space,
        z,
        cmap=cmap,
        norm=cm.colors.Normalize(vmin=0, vmax=1),
        shading="nearest",
        alpha=0.9,
    )
    ax.figure.colorbar(pcm, ax=ax)

In [ ]:
def plt_logistic(X_train, Y_train, model, set_w, set_b, pos, neg):
    fig, ax = plt.subplots(1, 2, figsize=(16, 4))
    layerf = lambda x: model.predict(x)
    plt_prob_1d(ax[0], layerf)

    layerf = lambda x: sigmoidnp(set_w @ x.reshape(1, 1) + set_b)
    plt_prob_1d(ax[1], layerf)
    ax[0].set_title("Tensorflow Model")
    ax[1].set_title("Numpy Model")
    for i in range(len(ax)):
        ax[i].scatter(X_train[pos], Y_train[pos], marker="x", s=80, c="r", label="y=1")
        ax[i].scatter(
            X_train[neg],
            Y_train[neg],
            marker="o",
            s=90,
            label="y=0",
            facecolors="none",
            edgecolors="b",
            lw=3,
        )
        ax[i].set_ylim(-0.08, 1.1)
        ax[i].set_xlim(-0.5, 5.5)
        ax[i].set_ylabel("y")
        ax[i].set_xlabel("x")
        ax[i].legend()
    plt.show()

In [ ]:
plt_logistic(X_train, Y_train, model, set_w, set_b, pos, neg)

In [ ]:
def plot_logistic_simple(ax, x_range, model, X, Y):
    """
    Plots P(y=1 | x) for a 1D logistic model and overlays training data.
    Includes contour (threshold) lines.
    """

    # --- x range for plotting ---
    x_plot = np.linspace(x_range[0], x_range[1], 300)

    # --- compute probabilities (SCALARS) ---
    p = np.array(
        [
            float(model.predict(np.array([[x]], dtype=np.float32), verbose=0))
            for x in x_plot
        ]
    )

    # --- probability curve ---
    ax.plot(x_plot, p, label="P(y=1 | x)", linewidth=2)

    # --- contour lines (probability thresholds) ---
    for level in [0.25, 0.5, 0.75]:
        ax.hlines(
            y=level,
            xmin=x_range[0],
            xmax=x_range[1],
            linestyles="dashed",
            linewidth=1,
            alpha=0.6,
            label="decision boundary (0.5)" if level == 0.5 else None,
        )

    # --- training points ---
    X_flat = X.reshape(-1)
    Y_flat = Y.reshape(-1)

    ax.scatter(
        X_flat[Y_flat == 1],
        Y_flat[Y_flat == 1],
        c="red",
        marker="x",
        s=80,
        label="y=1",
    )

    ax.scatter(
        X_flat[Y_flat == 0],
        Y_flat[Y_flat == 0],
        facecolors="none",
        edgecolors="blue",
        s=80,
        label="y=0",
    )

    # --- axes formatting ---
    ax.set_xlim(x_range)
    ax.set_ylim(-0.1, 1.1)
    ax.set_xlabel("x")
    ax.set_ylabel("Probability")
    ax.legend()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3))
plot_logistic_simple(ax, (0, 5), model, X_train, Y_train)
plt.show()